In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from operator import truediv

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
test_preds = pd.read_pickle("/mnt/data/Users/A-PROOF/Cecilia_S/models/fine_tuned/binary/test_set_pos_ell_sh_bl.pkl")

In [4]:
test_preds

,pad_sen_id,year,NotitieID,batch,annotator,background_sent,target_sent,text_raw,labels,text,len_text,icf-domains_incl_primary_10_epochs_lr3/best_model,icf-domains_incl_primary_10_epochs_lr5/best_model,icf-domains_incl_primary_half_none_10ep/best_model,index,institution,MDN,original_labels,pred_domains_eb_ap_mod1,pred_domains_eb_ap_mod2,secondary_preds,new_secondary_preds,icf-domains_combined_prim_sec,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_10_epochs,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_freeze_half_10ep/best_model,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_freeze_all_10ep,icf-domains_incl_primary_no_none_10ep/best_model,tmp_ellemijn/icf-domains_incl_primary_freeze_upto9_10ep,icf-domains_incl_primary_freeze_all_10ep/best_model,icf-domains_incl_primary_freeze_upto9_10ep/best_model,pred_best_model,pred_domains,confidence
0109003_0003,0109003_0003,2020,0109003,ergotherapie,edwin,False,False,"al ze even klaar mee is , wordt ze moe , zware schouders en gaat stoppen met activiteit , even naar buiten . Laad","[0, 0, 0, 1, 0, 0, 0, 0, 0]","al ze even klaar mee is , wordt ze moe , zware schouders en gaat stoppen met activiteit , even naar buiten . GPE",24.0,"[0, 0, 0, 1, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 0, 1, 0, 0, 1, 0, 0]","[0.09206172449106331, 0.09182581625967014, 0.09264681461737645, 0.18949445235758894, 0.09181080309283739, 0.09283210003120998, 0.1640260104330331, 0.0920809838641231, 0.09322129485309763]"
0119003_0001,0119003_0001,2020,0119003,ergotherapie,edwin,False,False,"beoordeling therapeut vermoeidheid neemt iets af , benauwdheid/pijn op de borst blijft nog wel bij inspannign","[1, 0, 0, 1, 0, 0, 0, 0, 0]","beoordeling therapeut vermoeidheid neemt iets af , benauwdheid/pijn op de borst blijft nog wel bij inspannign",16.0,"[0, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0.1270629643828882, 0.09080690478690641, 0.09026935305420021, 0.20904535839718577, 0.0906994251029122, 0.09054736720589183, 0.11713190350074809, 0.09333660665658648, 0.09110011691268084]"
0131001_0005,0131001_0005,2020,0131001,fysiotherapie,meskers,False,False,met 2 uur werken per dag . komt,"[0, 0, 1, 0, 0, 0, 0, 0, 0]",met 2 uur werken per dag . komt,8.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.09757922073314561, 0.09798776403548781, 0.21122954776422861, 0.09828643518001527, 0.09760572815291417, 0.09797025871428952, 0.10300088306641463, 0.09798244072327901, 0.09835772163022526]"
0131001_0011,0131001_0011,2020,0131001,fysiotherapie,meskers,False,False,"moe , zwaar werk vandaag gehad , ook iets langer dan 2 uur gewerkt , weinig kracht in handen . gaat","[0, 0, 1, 1, 0, 0, 0, 0, 0]","moe , zwaar werk vandaag gehad , ook iets langer dan 2 uur gewerkt , weinig kracht in handen . gaat",21.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.09456302393053685, 0.09540721445858541, 0.19185794505112364, 0.11800225257700431, 0.09476243908631385, 0.09476131067228999, 0.11284509533028643, 0.09622711346644709, 0.1015736054274123]"
0131001_0012,0131001_0012,2020,0131001,fysiotherapie,meskers,False,True,vragen of hij nog een weekje 2 uur per dag kan werken . evalueert,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",vragen of hij nog een weekje 2 uur per dag kan werken . evalueert,14.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.10120780370149568, 0.10137799709996509, 0.

In [5]:
test_preds['domain_pred'] = ""

In [6]:
test_preds['domain_gold'] = ""

In [7]:
for ri, row in test_preds.iterrows():
    if row['pred_domains'] == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ADM'
    if row['pred_domains'] == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ATT'
    if row['pred_domains'] == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'BER'
    if row['pred_domains'] == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ENR'
    if row['pred_domains'] == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ETN'
    if row['pred_domains'] == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'FAC'
    if row['pred_domains'] == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'INS'
    if row['pred_domains'] == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'MBW'
    if row['pred_domains'] == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        test_preds.loc[ri, 'domain_pred'] = 'STM'
    #if row['pred_domains'] == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
     #   test_preds.loc[ri, 'domain_pred'] = 'O'

In [8]:
for ri, row in test_preds.iterrows():
    if row['original_labels'] == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ADM'
    if row['original_labels'] == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ATT'
    if row['original_labels'] == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'BER'
    if row['original_labels'] == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ENR'
    if row['original_labels'] == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ETN'
    if row['original_labels'] == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'FAC'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'INS'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'MBW'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        test_preds.loc[ri, 'domain_gold'] = 'STM'
    #if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
     #   test_preds.loc[ri, 'domain_gold'] = 'O'

In [9]:
test_preds

,pad_sen_id,year,NotitieID,batch,annotator,background_sent,target_sent,text_raw,labels,text,len_text,icf-domains_incl_primary_10_epochs_lr3/best_model,icf-domains_incl_primary_10_epochs_lr5/best_model,icf-domains_incl_primary_half_none_10ep/best_model,index,institution,MDN,original_labels,pred_domains_eb_ap_mod1,pred_domains_eb_ap_mod2,secondary_preds,new_secondary_preds,icf-domains_combined_prim_sec,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_10_epochs,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_freeze_half_10ep/best_model,/data2/Documents/tmp_ellemijn/icf-domains_incl_primary_freeze_all_10ep,icf-domains_incl_primary_no_none_10ep/best_model,tmp_ellemijn/icf-domains_incl_primary_freeze_upto9_10ep,icf-domains_incl_primary_freeze_all_10ep/best_model,icf-domains_incl_primary_freeze_upto9_10ep/best_model,pred_best_model,pred_domains,confidence,domain_pred,domain_gold
0109003_0003,0109003_0003,2020,0109003,ergotherapie,edwin,False,False,"al ze even klaar mee is , wordt ze moe , zware schouders en gaat stoppen met activiteit , even naar buiten . Laad","[0, 0, 0, 1, 0, 0, 0, 0, 0]","al ze even klaar mee is , wordt ze moe , zware schouders en gaat stoppen met activiteit , even naar buiten . GPE",24.0,"[0, 0, 0, 1, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 0, 1, 0, 0, 1, 0, 0]","[0.09206172449106331, 0.09182581625967014, 0.09264681461737645, 0.18949445235758894, 0.09181080309283739, 0.09283210003120998, 0.1640260104330331, 0.0920809838641231, 0.09322129485309763]",,
0119003_0001,0119003_0001,2020,0119003,ergotherapie,edwin,False,False,"beoordeling therapeut vermoeidheid neemt iets af , benauwdheid/pijn op de borst blijft nog wel bij inspannign","[1, 0, 0, 1, 0, 0, 0, 0, 0]","beoordeling therapeut vermoeidheid neemt iets af , benauwdheid/pijn op de borst blijft nog wel bij inspannign",16.0,"[0, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0.1270629643828882, 0.09080690478690641, 0.09026935305420021, 0.20904535839718577, 0.0906994251029122, 0.09054736720589183, 0.11713190350074809, 0.09333660665658648, 0.09110011691268084]",ENR,
0131001_0005,0131001_0005,2020,0131001,fysiotherapie,meskers,False,False,met 2 uur werken per dag . komt,"[0, 0, 1, 0, 0, 0, 0, 0, 0]",met 2 uur werken per dag . komt,8.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.09757922073314561, 0.09798776403548781, 0.21122954776422861, 0.09828643518001527, 0.09760572815291417, 0.09797025871428952, 0.10300088306641463, 0.09798244072327901, 0.09835772163022526]",BER,
0131001_0011,0131001_0011,2020,0131001,fysiotherapie,meskers,False,False,"moe , zwaar werk vandaag gehad , ook iets langer dan 2 uur gewerkt , weinig kracht in handen . gaat","[0, 0, 1, 1, 0, 0, 0, 0, 0]","moe , zwaar werk vandaag gehad , ook iets langer dan 2 uur gewerkt , weinig kracht in handen . gaat",21.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.09456302393053685, 0.09540721445858541, 0.19185794505112364, 0.11800225257700431, 0.09476243908631385, 0.09476131067228999, 0.11284509533028643, 0.09622711346644709, 0.1015736054274123]",BER,
0131001_0012,0131001_0012,2020,0131001,fysiotherapie,meskers,False,True,vragen of hij nog een weekje 2 uur per dag kan werken . evalueert,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",vragen of hij nog een weekje 2 uur per dag kan werken . evalueert,14.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]","[0.1

In [10]:
y_pred = test_preds['domain_pred'].tolist()
y_true = test_preds['domain_gold'].tolist()

In [11]:
unique_labels = np.unique([y_true, y_pred])
cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred, labels=unique_labels),
    index=['true:{:}'.format(x) for x in unique_labels],
    columns=['pred:{:}'.format(x) for x in unique_labels]
)

cmtx

,pred:,pred:ADM,pred:ATT,pred:BER,pred:ENR,pred:ETN,pred:FAC,pred:INS,pred:MBW,pred:STM
true:,433,159,1,25,73,232,135,86,136,91
true:ADM,8,171,0,0,1,0,1,1,0,0
true:ATT,1,0,7,0,0,0,1,0,0,0
true:BER,1,0,0,6,0,0,0,0,0,0
true:ENR,3,0,0,0,50,0,0,0,1,0
true:ETN,12,1,0,0,0,70,1,0,0,0
true:FAC,1,1,0,0,0,0,83,0,0,0
true:INS,8,1,0,0,0,0,6,21,0,0
true:MBW,1,0,0,0,0,0,0,0,39,0
true:STM,4,0,0,0,0,0,0,0,0,93


In [12]:
cf = cmtx.to_numpy()

In [13]:
tp = np.diag(cf)

precision = list(map(truediv, tp, np.sum(cf, axis = 0)))
recall = list(map(truediv, tp, np.sum(cf, axis = 1)))
f1 = []

for p, r in list(zip(precision, recall)):
    f = (2*p*r)/(p+r)
    f1.append(f)

#print('Precision: {}\nRecall: {}\nF1 score: {}'.format(precision, recall, f1))

In [14]:
results = pd.DataFrame([precision,
                       recall,
                       f1],
                      columns=['O', 'ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM'],
                      index=['Precision','Recall', 'F1 score'])

results

,O,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
Precision,0.917373,0.513514,0.875000,0.193548,0.403226,0.231788,0.365639,0.194444,0.221591,0.505435
Recall,0.315828,0.939560,0.777778,0.857143,0.925926,0.833333,0.976471,0.583333,0.975000,0.958763
F1 score,0.469886,0.664078,0.823529,0.315789,0.561798,0.362694,0.532051,0.291667,0.361111,0.661922


In [15]:
results.to_pickle("/mnt/data/Users/A-PROOF/Cecilia_S/models/fine_tuned/best_model/results_hq2.pkl")

In [16]:
domains = ['O', 'ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']

for d, p, r, f in list(zip(domains, precision, recall, f1)):
    print('{}\nPrecision: {}\nRecall: {}\nF1 score: {}\n'.format(d, p, r, f))

O
Precision: 0.9173728813559322
Recall: 0.31582786287381476
F1 score: 0.4698860553445469

ADM
Precision: 0.5135135135135135
Recall: 0.9395604395604396
F1 score: 0.6640776699029126

ATT
Precision: 0.875
Recall: 0.7777777777777778
F1 score: 0.823529411764706

BER
Precision: 0.1935483870967742
Recall: 0.8571428571428571
F1 score: 0.3157894736842105

ENR
Precision: 0.4032258064516129
Recall: 0.9259259259259259
F1 score: 0.5617977528089888

ETN
Precision: 0.23178807947019867
Recall: 0.8333333333333334
F1 score: 0.36269430051813467

FAC
Precision: 0.3656387665198238
Recall: 0.9764705882352941
F1 score: 0.532051282051282

INS
Precision: 0.19444444444444445
Recall: 0.5833333333333334
F1 score: 0.2916666666666667

MBW
Precision: 0.2215909090909091
Recall: 0.975
F1 score: 0.3611111111111111

STM
Precision: 0.5054347826086957
Recall: 0.9587628865979382
F1 score: 0.6619217081850535

